# Model Tuning
Grid Search Hyperparameter Tuning

In [1]:

# Reloading
%reload_ext autoreload
%autoreload 2

import pandas as pd
print('pandas version:{}'.format(pd.__version__))
import numpy as np
print('numpy version:{}'.format(np.__version__))
import math
from datetime import datetime
from dgpylib import dg_athena
from dgpylib import dg_s3
import os
import re
from matplotlib.ticker import MaxNLocator
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import sys
sys.path.insert(0, '../Modelling')
import helper_function2 as hf

import sys
sys.path.insert(0, '../EDA')
import data_integrity_fixer as dint
import basicstatsandplotter as bsp
import data_imputer as di
import data_integrity_fixer as dif
import parameters

##for modelling
import h2o
print('h2o version:{}'.format(h2o.__version__))
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.tree import H2OTree
from h2o.tree import H2ONode
from h2o.tree import H2OSplitNode
from h2o.tree import H2OLeafNode
from h2o.estimators.kmeans import H2OKMeansEstimator

#import xgboost as xgb
import sklearn
import category_encoders as ce
print('sklearn version:{}'.format(sklearn.__version__))
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
from sklearn.preprocessing import PowerTransformer
import time
import parameters
from statsmodels.stats.outliers_influence import variance_inflation_factor

import math
from h2o.grid.grid_search import H2OGridSearch

%run 'functions.ipynb'

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100

pandas version:1.4.1
numpy version:1.22.3
h2o version:3.30.0.4
sklearn version:1.0.2


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
conn1 = dg_s3.Connect('dgdatadump/DS&CA/Pricing/02 Projects/2022/Tel_Reg_2.0')

data_df = conn1.read('processed_data/Tel_Reg_Final_Table_Combined_072022.csv')

In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1-internal" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 11.0.9.1-internal+0-adhoc..src, mixed mode)
  Starting server from /home/ubuntu/anaconda3/envs/ahmad_env/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqmuk7ed9
  JVM stdout: /tmp/tmpqmuk7ed9/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmpqmuk7ed9/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,"2 years, 3 months and 12 days !!!"
H2O_cluster_name:,H2O_from_python_ubuntu_3uvdxp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.97 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
path = 'GBM_cancellation_model_3mo_not_tuned'
# load the model
gbm =  h2o.import_mojo(path)

generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1663165349660_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,20766.0,5.0,5.0,5.0,23.0,32.0,28.36




ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.10419553375312073
RMSE: 0.3227933297841217
LogLoss: 0.3589074332104395
Mean Per-Class Error: 0.40604490975515906
AUC: 0.6368789097625362
AUCPR: 0.2391244087089311
Gini: 0.2737578195250725

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.13158438834810424: 


,,0,1,Error,Rate
0,0,121484.0,66364.0,0.3533,(66364.0/187848.0)
1,1,12379.0,14315.0,0.4637,(12379.0/26694.0)
2,Total,133863.0,80679.0,0.367,(78743.0/214542.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010003,0.251466,4.958576,4.958576,0.616962,0.473967,0.616962,0.473967,0.049599,0.049599,395.857557,395.857557
1,,2,0.020001,0.205957,2.562875,3.761005,0.318881,0.223546,0.467956,0.348786,0.025624,0.075223,156.287529,276.100458
2,,3,0.030003,0.189744,1.857593,3.126436,0.231128,0.196912,0.389001,0.298153,0.018581,0.093804,85.759326,212.643558
3,,4,0.040001,0.180898,1.888434,2.817007,0.234965,0.184965,0.350501,0.269863,0.018881,0.112684,88.843442,181.700742
4,,5,0.050037,0.175734,1.571581,2.567226,0.195541,0.178126,0.319422,0.251464,0.015771,0.128456,57.158087,156.722602
5,,6,0.100041,0.162275,1.546285,2.056922,0.192394,0.168006,0.255929,0.209749,0.077321,0.205777,54.628519,105.692209
6,,7,0.150003,0.154077,1.373630,1.829335,0.170911,0.157935,0.227612,0.192491,0.068630,0.274406,37.363032,82.933510
7,,8,0.200003,0.147619,1.216015,1.676009,0.151300,0.150816,0.208534,0.182072,0.060800,0.335206,21.601493,67.600863
8,,9,0.300007,0.137796,1.172879,1.508296,0.145933,0.142429,0.187667,0.168858,0.117292,0.452499,17.287902,50.829615
9,,10,0.400034,0.129518,1.090962,1.403943,0.135741,0.133562,0.174683,0.160032,0.109126,0.561624,9.096153,40.394305




Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-09-09 15:12:57,0.044 sec,0.0,0.330064,0.375646,0.500000,0.124423,1.000000,0.875577
1,,2022-09-09 15:12:58,0.595 sec,1.0,0.329098,0.373219,0.586107,0.188938,3.381207,0.482973
2,,2022-09-09 15:12:58,0.874 sec,2.0,0.328428,0.371794,0.587753,0.190056,3.494532,0.499842
3,,2022-09-09 15:12:58,1.144 sec,3.0,0.327902,0.370708,0.592064,0.195903,3.485629,0.537326
4,,2022-09-09 15:12:58,1.283 sec,4.0,0.327403,0.369690,0.596645,0.199700,3.941875,0.497646
5,,2022-09-09 15:12:58,1.374 sec,5.0,0.327011,0.368885,0.598724,0.201513,3.998031,0.483192
6,,2022-09-09 15:12:59,1.468 sec,6.0,0.326687,0.368218,0.600028,0.202128,3.680325,0.400374
7,,2022-09-09 15:12:59,1.561 sec,7.0,0.326385,0.367601,0.601697,0.203981,4.085211,0.425716
8,,2022-09-09 15:12:59,1.687 sec,8.0,0.326113,0.367018,0.603047,0.205448,4.215072,0.421824
9,,2022-09-09 15:12:59,1.831 sec,9.0,0.325885,0.366538,0.603772,0.206703,4.217810,0.452494



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,ONS_avg_age,594.795410,1.000000,0.119132
1,ManufacturerBrandCode,534.801819,0.899136,0.107116
2,ONS_%people_working_FT,474.414764,0.797610,0.095021
3,PlansAcceptedPast1YearCount,455.415924,0.765668,0.091215
4,PlanLiveCount,432.606842,0.727320,0.086647
5,ClientAccountDesc,368.058289,0.618798,0.073718
6,AppAge,257.012756,0.432103,0.051477
7,PurchasePrice,241.694000,0.406348,0.048409
8,PlansActivePast5yrCount,233.384216,0.392377,0.046745
9,ApplianceCode,228.895432,0.384831,0.045846


In [ ]:
data_df

In [8]:
conn_s3 = dg_s3.Connect('dgdatadump/DS&CA/Pricing/02 Projects/2022/Tel_Reg_2.0/Cancellation Model')

df_raw12 = conn_s3.read('TelReg_Canx_filt_12mo.csv')
# df_raw9 = conn_s3.read('TelReg_Canx_filt_9mo.csv')
# df_raw6 = conn_s3.read('TelReg_Canx_filt_6mo.csv')
# df_raw3 = conn_s3.read('TelReg_Canx_filt_3mo.csv')

In [9]:
df_raw12.columns.to_list()

['offertotalprice',
 'TotalPrice',
 'PricingVersionDescription',
 'PurchasePrice',
 'Fee',
 'PlanLiveCount',
 'PlansActivePast1yrCount',
 'PlansActivePast5yrCount',
 'PlansAcceptedPast1YearCount',
 'PlansAcceptedPast5YearCount',
 'PlansCancelledPast1YearCount',
 'PlansCancelledPast5YearCount',
 'ClaimPast1YearCount',
 'ClaimPast3YearCount',
 'ClaimPast5YearCount',
 'ClaimAmount1YearTotal',
 'ClaimAmount3YearTotal',
 'ClaimAmount5YearTotal',
 'GoodsColour',
 'ClientGroupDesc',
 'ClientAccountDesc',
 'RouteToMarketName',
 'ManufacturerGuaranteeLabourMonths',
 'ManufacturerGuaranteePartsMonths',
 'ApplianceCode',
 'ClientGroupCode',
 'ClientCode',
 'ManufacturerBrandCode',
 'price_diff',
 'price_band',
 'HasPrevRTPPrice',
 'AppAge',
 'relFee',
 'relFeeInc',
 'cancelflag',
 'HPI_AveragePrice',
 'HPI_SalesVolume',
 'ONS_avg_age',
 'ONS_avg_distance_travelled_to_work(km)',
 'ONS_%people_working_FT',
 'ONS_%people_working_49+',
 'ONS_%people_working_sum',
 'ONS_avg_household_size',
 'ONS_n_ho

In [ ]:
# read feature importance performance pickle
feat_imp_performance_df = pd.read_pickle("feat_imp_performance_df.pkl")
# read model performance pickle
performance_df = pd.read_pickle("performance_df.pkl")

In [ ]:
# for 12/9/6/3-month journey
idx = 3
predictors = performance_df.loc[idx, 'feats_stored_gbm']
response = 'cancelflag'
dataset = 'df_raw12'
print(f'dataset: {dataset}')
print(f'predictors: {predictors}')
print(f'number of predictors: {len(predictors)}')

In [ ]:
h2o.init()

In [ ]:
# load best model
path = 'GBM_cancellation_model_12mo'
# load the model
best_model =  h2o.import_mojo(path)

In [ ]:
df = df_raw12.copy()

In [ ]:
# convert columns to factors
for i in df_raw12.columns:
    df[i] = df_raw12[i].astype('category')
    
#train test split
x_train, x_test, y_train, y_test = train_test_split(df[predictors], df[response], test_size = 0.2, random_state = 1)
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

# preserve training & test data's index to be used as a key to merge prediction probability with main dataset 
xtrain_index = x_train.index
xtest_index = x_test.index

# convert to h2o dataframe format
hf_train = h2o.H2OFrame(train)
hf_test = h2o.H2OFrame(test)
# hf_df = h2o.H2OFrame(df)

# format target to fit the model
hf_train['cancelflag']=hf_train['cancelflag'].asfactor()
hf_test['cancelflag']=hf_test['cancelflag'].asfactor()

In [ ]:
#get the parameters from the Random grid search model and modify them slightly
params = best_model.params
new_params = {"nfolds":5, "model_id":None, "training_frame":None, "validation_frame":None, 
              "response_column":None, "ignored_columns":None}
for key in new_params.keys():
    params[key]['actual'] = new_params[key] 
gbm_best = H2OGradientBoostingEstimator()
for key in params.keys():
    if key in dir(gbm_best) and getattr(gbm_best,key) != params[key]['actual']:
        setattr(gbm_best,key,params[key]['actual'])

In [ ]:
# train GBM model
start = time.time()
gbm_best.train(x=predictors, y=response, training_frame=hf_train)
end = time.time()
print(end - start)

In [ ]:
# cross validation
print(gbm_best.cross_validation_metrics_summary())

In [ ]:
# df = df_raw12.copy()
# hf_df = h2o.H2OFrame(df)

# hf_train, hf_valid, hf_test = hf_df.split_frame(
#     ratios=[0.6,0.2], 
#     seed=1234, 
#     destination_frames=['train.hex','valid.hex','test.hex']
# )

In [ ]:
y_test = hf_test.as_data_frame()
y_test=y_test['cancelflag']

# classification report
y_true = y_test
y_pred = gbm_best.predict(hf_test)
y_pred = y_pred.as_data_frame()

print(classification_report(y_true, y_pred['predict'], labels=[0,1]))

In [ ]:
model_perf_stats(gbm_best, hf_train, hf_test)

In [ ]:
# path = 'GBM_cancellation_model_12mo'
# gbm.save_mojo(path)

In [ ]:
# variable importance
varimp = gbm_best.varimp(use_pandas=True).sort_values('scaled_importance', ascending=False)
feats_stored_gbm = list(varimp['variable'])
#perf metrics
perf_metrics = model_perf_stats(gbm_best, hf_train, hf_test)
# classification report
classif_rep = classification_report(y_true, y_pred['predict'], labels=[0,1])
# AUC of cross-validated holdout predictions
xval_auc =  f'AUC of cross-validated holdout predictions: {best_model.auc(xval=True)}'
# cross-validation metrics summary
xval_metrics = best_model.cross_validation_metrics_summary()

In [ ]:
# assign value to row
tuned_performance_df.loc[idx,1:] = [varimp, feats_stored_gbm, perf_metrics, classif_rep, xval_auc, xval_metrics]

In [ ]:
# # 12 month
# for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

#     print(tuned_performance_df.loc[idx,j])

gbm_best12 = best_model

In [ ]:
# # 9 month
# for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

#     print(tuned_performance_df.loc[idx,j])

# gbm_best9 = best_model

In [ ]:
# # 6 month
# for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

#     print(tuned_performance_df.loc[idx,j])
    
# gbm_best6 = best_model

In [ ]:
# # 3 month
# for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

#     print(tuned_performance_df.loc[idx,j])

# gbm_best3 = best_model

In [ ]:
# print performance metrics for all cancellation cohorts
for i in range(len(tuned_performance_df)):
    for j in ['name','feats_stored_gbm','perf_metrics','classif_rep', 'xval_auc']:

        print(tuned_performance_df.loc[i,j])

In [ ]:
# print cross-validated performance metrics for all cancellation cohorts
for i in range(len(tuned_performance_df)):
    print(tuned_performance_df.loc[i,'name'])
    print(tuned_performance_df.loc[i,'xval_metrics'])
        

In [ ]:
# # Extract feature interactions:
# feature_interactions = gbm_best.feature_interaction()

## Save Performance

In [ ]:
# # save performance metric to pickle
# pd.to_pickle(tuned_performance_df, "tuned_performance_df.pkl")

## Save Models

In [ ]:
# # saving down the model
# path = 'GBM_cancellation_model_3mo'
# gbm_best3.save_mojo(path)

# path = 'GBM_cancellation_model_6mo'
# gbm_best6.save_mojo(path)

# path = 'GBM_cancellation_model_9mo'
# gbm_best9.save_mojo(path)

path = 'GBM_cancellation_model_12mo'
gbm_best12.save_mojo(path)


## Resampling

In [ ]:
# df = df_raw3
# class_counts = df['cancelflag'].value_counts()
# class_counts

In [ ]:
# minority_label = 1
# majority_label = 0
# class_counts[minority_label] # count of minority class 
# sampled_df = imbalanced_undersample(df, 'cancelflag', minority_label, majority_label)
# sampled_df['cancelflag'].value_counts()